<a href="https://colab.research.google.com/github/LeoPiresDeSouza/DSAcademy-IAGenerativaProcessamentoLinguagemnatural/blob/master/Projeto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>IA Generativa e LLMs Para Processamento de Linguagem Natural</font>
## <font color='blue'>Projeto 3</font>
## <font color='blue'>Fine-Tuning de LLM Open-Source Para App de Assistente Jurídico</font>

## Instalando e Carregando Pacotes

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.7 MB/s eta 0:00:00


In [2]:
# Instalando os pacotes
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 35.3 MB/s eta 0:00:00


In [3]:
# Imports
import numpy as np
import nltk
import shutil
import evaluate
import transformers
import datasets
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [5]:
# Remove as pastas, se já existirem. Isso evita erros ao executar o jupyter a partir da segunda vez em diante
try:
    shutil.rmtree('logs_treino')
    shutil.rmtree('resultados_treino')
    shutil.rmtree('modelo_salvo')
except:
    print('As pastas não existem ou já foram removidas!')

As pastas não existem ou já foram removidas!


## Carregando os Dados

In [6]:
# Define o nome do arquivo
nome_arquivo = "dataset.csv"

load-dataset é uma biblioteca do Hugging Face

In [7]:
# Carrega os dados
dsa_dataset = load_dataset('csv', data_files = nome_arquivo, delimiter = ',')

Generating train split: 0 examples [00:00, ? examples/s]

O pacote permite dividir o arquivo de entrada em duas massas diferentes. Uma será utilizada para treino e outr será utilizada para testear o modelo treinado.

In [8]:
# Divisão em treino e teste com proporção 80/20
dsa_dataset = dsa_dataset["train"].train_test_split(test_size = 0.2)

In [ ]:
# Visualiza o formato do dataset
dsa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 749
    })
})

## Tokenizador e LLM Open-Source

https://huggingface.co/google/flan-t5-base

legacy = False é encessário para as novas versões do tokenizador para que ele trabalhe de acordo com os novos procedimentos.

Caso na documentação do modelo não cite o tokenizador, é necessário buscar na documentação do tokenizador.

In [9]:
# Carrega o tokenizador
tokenizador = T5Tokenizer.from_pretrained("google/flan-t5-base", legacy = False)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# Visualiza o tokenizador
tokenizador

T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '

In [11]:
# Carrega o LLM pré-treinado
modelo = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [12]:
# Visualiza a arquitetura do LLM
modelo

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

Cria um pipeline para concatenar o tokenizador e o modelo. Biblioteca transformers do Hugging face.

In [13]:
# Data collator para concatenar tokenizador e modelo
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizador, model = modelo)

In [ ]:
# Visualiza o data collator
data_collator

DataCollatorForSeq2Seq(tokenizer=T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<

## Pré-Processamento

Estratégia de prompt engeneering.

In [16]:
# Todos os inputs vão receber como prefixo: "answer the question" (responda a questão)
prefix = "answer the question: "

In [14]:
# Função de pré-processamento
def dsa_fn_preprocess(data):

    # Concatena o prefixo a cada pergunta na lista de perguntas fornecida em data["question"]
    inputs = [prefix + doc for doc in data["question"]]

    # Usa o tokenizer para converter as perguntas processadas em tokens com um comprimento máximo de 128,
    # truncando as que forem mais longas. Um max-length maior pode trazer maior precisão, mais aumenta o custo
    # do treinamento pois consome mais recursos.

    model_inputs = tokenizador(inputs, max_length = 128, truncation = True)

    # Tokeniza as respostas fornecidas em data["answer"] com um comprimento máximo de 512,
    # truncando as que forem mais longas
    labels = tokenizador(text_target = data["answer"], max_length = 512, truncation = True)

    # Adiciona os tokens das respostas como rótulos no dicionário de entradas do modelo
    model_inputs["labels"] = labels["input_ids"]

    # Retorna o dicionário processado contendo tanto as entradas tokenizadas quanto os rótulos
    return model_inputs

A função map mapeia o conunto de dados para a função criada anteriormente.

In [17]:
# Aplica a função de pré-processamento ao dataset gerando o dataset tokenizado
dsa_dataset_tokenized = dsa_dataset.map(dsa_fn_preprocess, batched = True)

Map:   0%|          | 0/2993 [00:00<?, ? examples/s]

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

In [18]:
# Visualiza o dataset tokenizado
dsa_dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 749
    })
})

In [19]:
dsa_dataset_tokenized['train']['question'][0]

"Q: What rights for I have to sue the SSA FOR SEVERE SADISTICALLY MOTIVATED HARASSMENT, AND THE ILLEGAL REMOVAL OF MY Job?. Removed my job, removed my payback to them stating that I didn't? I just got done paying them back! Massachusetts sent letters to NEW YORK STATE TEMP & DISABILITY OFFICE WHO HAS NOW STARTED HARASSING ME DENYING ME MY BASIC HUMAN RIGHTS TO SAFE SHELTER. WATER? IM A WHISTLE-BLOWER MASS COMMITTED TAX EMBEZZLEMENT AGAINST FEDERAL GOVERNMENT MONEY I NEVER RECEIVED ON MY 1099 AND I TOLD IRS FORM 3949-A & 211 TO RECEIVE AWARD, SINCE SOMEONES BEEN ILLEGALLY ALTERING ALL MY DOCUMENTS AND SHUTTING DOWN MY BENEFITS , AS IF THEY WANT ME DEAD OR PREOCCUPIED SO I CANT FIGHT BACK! I NEED AN ATTORNEY IVE NO ONE HERE IN NY ! SCARED ALL THE TIME SOMEONE WILL HURT ME... NO ONE WILL HELP ME , CANT LET THEM WIN! IVE GOT EVIDENCE OF 608 GILES REMOVED FROM MY CELL PHONE CYBER SECURITY CO. GEO LOCATION SAYS ' BOSTON'! I FEEL STALKED BY THESE LUNATICS, SSI REPEATEDLY ASKING WHOSE MY DAD? 

In [20]:
dsa_dataset_tokenized['train']['answer'][0]

"A:Your situation sounds incredibly challenging and distressing. If you believe you are experiencing harassment and illegal actions by the Social Security Administration (SSA) or any other entity, it's important to take action to protect your rights. Firstly, consider documenting all instances of harassment, illegal removal of your job, and any other irregularities. This documentation is crucial for any legal action you may take. Seeking legal representation is a critical next step. An attorney experienced in social security law, employment law, and whistleblower protection can provide guidance and represent your interests. They can help you understand your rights and the legal avenues available to address your grievances. You may also have grounds to file a complaint with the SSA's Office of the Inspector General (OIG). The OIG investigates allegations of misconduct within the SSA. Your evidence and documentation will be vital in this process. Remember, you have the right to seek lega

In [21]:
dsa_dataset_tokenized['train']['input_ids'][0]

[1525,
 8,
 822,
 10,
 1593,
 10,
 363,
 2166,
 21,
 27,
 43,
 12,
 2629,
 15,
 8,
 3,
 26787,
 5652,
 180,
 22516,
 427,
 180,
 6762,
 13582,
 23936,
 5121,
 7109,
 5494,
 8230,
 11430,
 454,
 21159,
 4256,
 11810,
 6,
 3430,
 1853,
 3,
 10781,
 8579,
 4090,
 4083,
 5365,
 21712,
 3347,
 283,
 476,
 7575,
 58,
 5,
 10002,
 26,
 82,
 613,
 6,
 3641,
 82,
 726,
 1549,
 12,
 135,
 3,
 17211,
 24,
 27,
 737,
 31,
 17,
 58,
 27,
 131,
 530,
 612,
 3788,
 135,
 223,
 55,
 9777,
 1622,
 5487,
 12,
 8747,
 3,
 476,
 2990,
 439,
 5097,
 6048,
 3,
 20050,
 345,
 3,
 184,
 3,
 15438,
 27474,
 3,
 15316,
 8906,
 20989,
 454,
 3291,
 3,
 12038,
 3,
 19481,
 11430,
 454,
 21159,
 4256,
 2365,
 7934,
 3396,
 12056,
 2365,
 7934,
 283,
 476,
 8145,
 21192,
 3,
 17861,
 9312,
 3,
 27262,
 134,
 3001,
 1]

In [22]:
dsa_dataset_tokenized['train']['labels'][0]

[71,
 10,
 21425,
 1419,
 2993,
 3,
 5828,
 4421,
 11,
 19285,
 53,
 5,
 156,
 25,
 857,
 25,
 33,
 8154,
 23556,
 11,
 6016,
 2874,
 57,
 8,
 2730,
 3684,
 6863,
 41,
 26787,
 61,
 42,
 136,
 119,
 10409,
 6,
 34,
 31,
 7,
 359,
 12,
 240,
 1041,
 12,
 1822,
 39,
 2166,
 5,
 3,
 23559,
 6,
 1099,
 1708,
 53,
 66,
 10316,
 13,
 23556,
 6,
 6016,
 4614,
 13,
 39,
 613,
 6,
 11,
 136,
 119,
 22085,
 2197,
 5,
 100,
 7192,
 19,
 4462,
 21,
 136,
 1281,
 1041,
 25,
 164,
 240,
 5,
 1610,
 1765,
 1281,
 6497,
 19,
 3,
 9,
 2404,
 416,
 1147,
 5,
 389,
 4917,
 1906,
 16,
 569,
 1034,
 973,
 6,
 4311,
 973,
 6,
 11,
 23585,
 4672,
 3321,
 1711,
 54,
 370,
 4864,
 11,
 4221,
 39,
 3984,
 5,
 328,
 54,
 199,
 25,
 734,
 39,
 2166,
 11,
 8,
 1281,
 18836,
 7,
 347,
 12,
 1115,
 39,
 20214,
 663,
 7,
 5,
 148,
 164,
 92,
 43,
 9808,
 12,
 1042,
 3,
 9,
 10394,
 28,
 8,
 3,
 26787,
 31,
 7,
 2126,
 13,
 8,
 19773,
 2146,
 41,
 667,
 8834,
 137,
 37,
 411,
 8834,
 9127,
 7,
 20265,
 13,
 28742,
 44

## Definindo a Métrica de Avaliação do Modelo

Rouge é uma métrica amplemanete utilizada para avaliar a  qualidade de resumos de textos produzidos por máquinas em comparação com um conunto de resumos de referência produzidos por humanos.

O módulo punkt é necessário para que a métrica rouge funcione.

In [23]:
# O pacote "punkt" é específico para a tarefa de tokenização, que envolve a divisão de um texto
# em uma lista de sentenças
nltk.download("punkt", quiet = True)

True

O pacote evaluate trm várias métricas para avaliação de treinamentos de aprendizado de máquina.

In [24]:
# Define a métrica
metric = evaluate.load("rouge")

In [26]:
# Função de cálculo das métricas
# O argumento são as previsões do modelo, que nada mais são do que os textos de saída,
# ou seja a resposta às perguntas.

def dsa_calcula_metricas(eval_preds):

    # Desempacota as predições e os rótulos do argumento eval_preds
    previsoes, labels = eval_preds

    # Substitui todos os valores diferentes de -100 em labels pelo ID do token de preenchimento.
    # O -100 é colocado pelo tokenizador do T5 para preencher o token se ele for menor
    # que o tamanho máximo definido. Isso traz-se da documentação do tokenizador.

    labels = np.where(labels != -100,
                      labels,
                      tokenizador.pad_token_id)

    # Decodifica as previsões para texto, ignorando tokens especiais
    previsoes_decodificadas = tokenizador.batch_decode(previsoes,
                                                       skip_special_tokens = True)

    # Decodifica os labels para texto, ignorando tokens especiais
    labels_decodificados = tokenizador.batch_decode(labels,
                                                    skip_special_tokens = True)

    # Adiciona uma nova linha após cada sentença para as previsões decodificadas,
    # preparando-as para a avaliação ROUGE
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in previsoes_decodificadas]

    # Adiciona uma nova linha após cada label para as previsões decodificadas,
    # preparando-os para a avaliação ROUGE
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in labels_decodificados]

    # Calcula a métrica ROUGE entre as previções e os rótulos decodificados, utilizando um stemmer
    resultado = metric.compute(predictions = previsoes_decodificadas,
                               references = labels_decodificados,
                               use_stemmer = True)

    # Retorna o resultado da métrica ROUGE
    return resultado

In [27]:
# Define argumentos de treino
dsa_training_args = Seq2SeqTrainingArguments(output_dir = "resultados_treino",
                                             evaluation_strategy = "epoch",
                                             learning_rate = 3e-4,
                                             logging_dir = "logs_treino",
                                             logging_steps = 1,
                                             per_device_train_batch_size = 4,
                                             per_device_eval_batch_size = 2,
                                             weight_decay = 0.01,
                                             save_total_limit = 3,
                                             num_train_epochs = 3,
                                             predict_with_generate = True,
                                             push_to_hub = False)

In [28]:
# Define o trainer
dsa_trainer = Seq2SeqTrainer(model = modelo,
                             args = dsa_training_args,
                             train_dataset = dsa_dataset_tokenized["train"],
                             eval_dataset = dsa_dataset_tokenized["test"],
                             tokenizer = tokenizador,
                             data_collator = data_collator,
                             compute_metrics = dsa_calcula_metricas)

## Treinamento do Modelo

In [1]:
%%time
dsa_trainer.train()

NameError: name 'dsa_trainer' is not defined

In [ ]:
# Salva o modelo
dsa_trainer.save_model('modelo_salvo')

**ROUGE-1** mede a sobreposição de unigramas (palavras individuais).

**ROUGE-2** mede a sobreposição de bigramas (pares de palavras consecutivas).

**ROUGE-L** mede a sobreposição da subsequência mais longa comum entre o resumo gerado e o resumo de referência. Isso leva em conta a ordem das palavras, mas permite lacunas. Valores mais altos indicam melhor desempenho. ROUGE-L é calculado com base na similaridade entre as sequências, levando em consideração a precisão, o recall e a média harmônica entre eles.

Valores mais altos de ROUGE indicam uma maior semelhança entre o resumo gerado e o resumo de referência, o que geralmente é interpretado como uma indicação de melhor qualidade do resumo. No entanto, é importante lembrar que nenhuma métrica pode capturar completamente a qualidade de um resumo e é útil complementar a avaliação com análises qualitativas ou outras métricas.

## Deploy e Uso do Modelo

In [ ]:
# Carrega o tokenizador final salvo em disco
tokenizador_dsa_final = AutoTokenizer.from_pretrained('modelo_salvo')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Carrega o modelo final salvo em disco
modelo_dsa_final = AutoModelForSeq2SeqLM.from_pretrained('modelo_salvo')

In [ ]:
# Visualiza a arquitetura do modelo
modelo_dsa_final

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# Texto de entrada
texto_entrada = "Can I move out of state with my children if I have a custody agreement in that state?"

In [ ]:
# Tokeniza o input
texto_entrada_tokenizado = tokenizador_dsa_final(texto_entrada, return_tensors = "pt").input_ids

In [ ]:
# Gera a saída (previsão do modelo)
texto_saida_tokenizado = modelo_dsa_final.generate(texto_entrada_tokenizado,
                                                   max_length = 50,
                                                   temperature = 0.4,
                                                   do_sample = True)

In [ ]:
# Visualiza
texto_saida_tokenizado

tensor([[    0,    71, 16701,  2791,    19,     3,     9, 13595, 11293,  2791,
           344,   192,  1362,     5,   156,    25,    43,    46,  2791,    16,
           286,    24,  1250,    25,    12,   888,    91,    13,   538,    28,
            39,   502,     6,    25,   164,    36,     3,   179,    12,   888,
            91,    13,   538,    28,    39,   502,     5,   611,     6,     3]])

In [ ]:
# Decode da saída
texto_saida = tokenizador_dsa_final.decode(texto_saida_tokenizado[0],
                                           skip_special_tokens = True)

In [ ]:
# Resultado
print("Pergunta:", texto_entrada)
print("Resposta:", texto_saida)

Pergunta: Can I move out of state with my children if I have a custody agreement in that state?
Resposta: A custody agreement is a legally binding agreement between two parents. If you have an agreement in place that allows you to move out of state with your children, you may be able to move out of state with your children. However, 


In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [ ]:
#%watermark -v -m

In [ ]:
#%watermark --iversions

# Fim